## An over-simplified model to predict:
### 1. ability of each student
### 2. attributes of each problem

In [1]:
import numpy as np
import tensorflow as tf
import time

In [2]:
'''
these parameters means: 
number of students
number of problems
number of latent factors
ratio of missing values and complete data
number of missing values
hyper parameters 1, to restrict the sum of elements in each problem to be one
hyper parameters 2, to restrict elements in each problem to be less than one  
'''
n_student = 100
n_prob = 80
n_latent = 5
ratio = 0.1
n_zeros = int(n_student*n_prob* ratio)
lamb1 = 1.
lamb2 = 1.

### Before collecting real data, I produce ARTIFICIAL data:

In [3]:
# In this matrix, each row correspond to one problem
p_mat = np.random.uniform(0,1,(n_prob,n_latent))
p_mat.shape

(80, 5)

In [4]:
# In this matrix, each row correspond to one student
s_mat = np.random.randint(10, size=(n_student,n_latent))
s_mat.shape

(100, 5)

In [5]:
# proj_mat defines positions to drop a ratio of data randomly
# (optional)
proj_mat = np.ones(n_student*n_prob)
proj_mat[:n_zeros] = 0
np.random.shuffle( proj_mat )
proj_mat = proj_mat.reshape([n_student, n_prob])

In [6]:
# scores collected in this imaginary exam:
scores = np.multiply( np.dot(s_mat, np.transpose(p_mat) ), proj_mat)  
scores.shape

(100, 80)

### Use tensorflow(TF) to construct model:
### 1. If we actually need neural network model, we use TF.
### 2. If not, I will rewrite this without tensorflow when I have time.

In [7]:
sess = tf.Session()

In [8]:
class recomend:
    def __init__(self, n_student, n_prob, n_latent, ratio, lamb1, lamb2):
        self.ones = tf.constant(np.ones([n_prob,1]), dtype=tf.float32)
        self.n_student = n_student
        self.n_prob = n_prob
        #self.lamb = lamb
        self.scores = tf.placeholder(dtype = tf.float32 ,shape = [n_student, n_prob])
        self.proj_ = tf.placeholder(dtype = tf.float32 ,shape = [n_student, n_prob])
        self.s_ = tf.get_variable("student", shape=[n_student, n_latent])
        self.p_ = tf.get_variable("problem", shape=[n_prob, n_latent])
        self.predict = tf.mul( tf.matmul(self.s_, tf.transpose(self.p_) ), self.proj_) 
        
        loss_0 = tf.reduce_sum(tf.square(self.predict-self.scores) )/(n_student*n_prob* (1-ratio) )
        loss_1 = lamb1 * tf.reduce_sum(tf.square(tf.reduce_sum(self.p_, reduction_indices = 1) - self.ones) )
        loss_2 = lamb2 * tf.reduce_sum(tf.pow(self.p_, 8) )/(n_student*n_prob* (1-ratio) )
        self.loss = loss_0 + loss_1 + loss_2

In [9]:
model = recomend(n_student, n_prob, n_latent, ratio, lamb1, lamb2)

In [10]:
sess.run(tf.initialize_all_variables() )

In [11]:
train_op = tf.train.GradientDescentOptimizer(1e-3).minimize(model.loss)
#train_op = tf.train.AdamOptimizer(1e-4).minimize(model.loss)

In [ ]:
## start training:
delta = 10
loss_old = 1e3
steps = 0
t0 = 0
while loss_0 > 5e-1:
    sess.run(train_op, feed_dict = {model.scores:scores, model.proj_: proj_mat })
    loss_new = sess.run(model.loss, feed_dict = {model.scores:scores,
                                    model.proj_: proj_mat })
    delta = loss_0 - loss_1
    loss_old = loss_new
    steps +=1
    if steps%5000 ==0:
        t1 = time.time() - t0
        print(steps, loss_new, t1)
        t0 = time.time()

In [13]:
## final loss
loss_new

0.5198372

In [14]:
delta

1.1920929e-07

In [15]:
ans_p = sess.run(model.p_, feed_dict = {model.scores:scores, model.proj_: proj_mat })
ans_s = sess.run(model.s_, feed_dict = {model.scores:scores, model.proj_: proj_mat })

In [16]:
print(np.max(ans_p))
print(np.max(ans_s))
print(np.max(p_mat))
print(np.max(s_mat))

1.67534
11.2197
0.998288734857
9


### The matrix "scores" corresponds to the actual scores students get,  each row corresponds to one student

In [17]:
scores

array([[ 12.80680776,   8.43003059,   4.13264195, ...,  14.77383256,
          8.50550194,   0.        ],
       [ 13.39813329,   9.19511565,   7.3082714 , ...,   0.        ,
         10.499488  ,   8.99193504],
       [ 11.96497792,   7.99475783,   9.51668089, ...,   9.80643314,
         12.78025442,   8.37974439],
       ..., 
       [ 12.9880673 ,   7.32100781,  10.20719712, ...,  14.29542246,
          0.        ,   8.64498603],
       [ 13.11857865,   7.7657547 ,   9.24343606, ...,  15.86665384,
         12.10683107,   8.52417916],
       [  0.        ,  13.369907  ,  12.01829756, ...,  20.43683902,
         17.42720044,  14.02801545]])

### The matrix "predict" corresponds to the scores predicted by the model

In [18]:
predict = sess.run(model.predict, feed_dict = {model.scores:scores, model.proj_: proj_mat })

array([[ 12.64943314,   7.38609791,   4.88772869, ...,  13.9958477 ,
          9.47811222,   0.        ],
       [ 13.48714733,   9.46942711,   6.95884275, ...,   0.        ,
         10.24669075,   9.32217979],
       [ 11.98147964,   8.16967869,   9.22065926, ...,   9.99115658,
         12.56333351,   8.55237675],
       ..., 
       [ 13.08638382,   7.53767014,  10.00169468, ...,  14.36231518,
          0.        ,   8.8303442 ],
       [ 13.29052734,   7.94151449,   8.9777174 , ...,  16.02168465,
         12.02047634,   8.74792957],
       [  0.        ,  13.06660366,  12.04007149, ...,  20.25844955,
         17.75388718,  13.78044796]], dtype=float32)

## The model seems to work, and this is not the best we can do yet~  